In [16]:
import torch 
import time
from scipy.sparse import random
import numpy as np

def mv(matrix, vector):
  t = time.time()

  for _ in range(100):
    ans = torch.mv(matrix, vector)

  return time.time() - t

def mv_gpu(matrix, vector):

  torch.cuda.synchronize()

  start = torch.cuda.Event(enable_timing=True)
  end = torch.cuda.Event(enable_timing=True)

  start.record()

  for _ in range(100):
    ans = torch.mv(matrix, vector)
  end.record()

  torch.cuda.synchronize()

  return start.elapsed_time(end)

M = 2 ** 13
N = 2 ** 13

print(f'{M} x {N}')
print(M*N*4)

matrix = torch.tensor(random(M, N, density=0.5).A).float()
vector = torch.rand([N,])

print('cpu dense', mv(matrix, vector) * 1000)
print('gpu dense', mv_gpu(matrix.cuda(), vector.cuda()))

matrix = matrix.to_sparse_csr()

print('cpu sparse csr', mv(matrix, vector) * 1000)
print('gpu sparse csr', mv_gpu(matrix.cuda(), vector.cuda()))

8192 x 8192
268435456
cpu dense 1040.5998229980469
gpu dense 100.80118560791016
cpu sparse csr 3605.8313846588135
gpu sparse csr 161.3987579345703


In [28]:
import torch 
import time
from scipy.sparse import random
import numpy as np

def mv(A, B):
  t = time.time()

  for _ in range(100):
    ans = torch.matmul(A, B)

  return time.time() - t

def mv_gpu(A, B):

  torch.cuda.synchronize()

  start = torch.cuda.Event(enable_timing=True)
  end = torch.cuda.Event(enable_timing=True)

  start.record()

  for _ in range(100):
    ans = torch.matmul(A, B)
  end.record()

  torch.cuda.synchronize()

  return start.elapsed_time(end)

M = 2 ** 10
N = 2 ** 10

print(f'{M} x {N}')
print(M*N*4)

matrix = torch.tensor(random(M, N, density=0.01).A).float()

print('cpu dense', mv(matrix, matrix) * 1000)
print('gpu dense', mv_gpu(matrix.cuda(), matrix.cuda()))

matrix_sparse = matrix.to_sparse_csr()

print('cpu sparse csr', mv(matrix_sparse, matrix) * 1000)
print('gpu sparse csr', mv_gpu(matrix_sparse.cuda(), matrix.cuda()))

1024 x 1024
4194304
cpu dense 1422.1227169036865
gpu dense 82.73241424560547
cpu sparse csr 172.35565185546875
gpu sparse csr 17.647008895874023
